# 특성 공학과 규제

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/3-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 데이터 준비

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns

import datetime
from datetime import date

In [ ]:
from matplotlib import rc
from matplotlib import font_manager
import matplotlib.font_manager as fm

fm.get_fontconfig_fonts()
font_path =  "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

# 데이터 프레임 출력시 요약x. 전체 출력o
pd.set_option('display.max_rows', None)

# 데이터 불러오기

In [ ]:
df_stock = pd.read_csv("C:\\Users\\Hi\\JupyterProjects\\JupyterML\\Covid\\data\\covid19_kor\\주식시장_일별__20220415170553.csv", encoding='euc-kr')
df_covid = pd.read_csv("C:\\Users\\Hi\\JupyterProjects\\JupyterML\\Covid\\data\\covid19_kor\\서울특별시 코로나19 자치구별 확진자 발생동향.csv", encoding='euc-kr')
df_vaccine = pd.read_csv("C:\\Users\\Hi\\JupyterProjects\\JupyterML\\Covid\\data\\covid19_kor\\서울특별시 코로나19 백신 예방접종 현황.csv", encoding='euc-kr')
df_distance = pd.read_csv("C:\\Users\\Hi\\JupyterProjects\\JupyterML\\Covid\\data\\covid19_kor\\지역별 거리두기.csv")

# 데이터 확인

In [ ]:
df_stock.head()

In [ ]:
df_covid.head()

In [ ]:
df_vaccine.head()

In [ ]:
df_distance.head()

In [ ]:
df_covid.info()

In [ ]:
df_stock.info()

In [ ]:
df_vaccine.info()

In [ ]:
df_distance.info()

# Keep Distance

In [ ]:
# 오미크론 우세종 된 이후 데이터 제거
df_distance = df_distance[df_distance['시도명'] == '서울']

In [ ]:
df_distance.loc[12]=['서울', '2020-02-05','2020-03-21', 0, 0.0]
df_distance.loc[14]=['서울','2020-05-06','2020-08-16', 1, 1.0]
df_distance.loc[193]=['서울', '2021-10-18','2022-02-17', 4, 4.0]

In [ ]:
df_distance = df_distance.drop(['시도명'], axis = 1)

In [ ]:
df_distance # 확인용 코드

In [ ]:
df_distance = df_distance.sort_index(ascending=True)

In [ ]:
df_distance # 확인용 코드

In [ ]:
df_distance = df_distance.reset_index()

In [ ]:
df_distance # 확인용 코드

In [ ]:
df_distance = df_distance.drop(['index'], axis = 1)

In [ ]:
df_distance # 확인용 코드

In [ ]:
len(df_distance) # 확인용 코드

In [ ]:
df_distance.loc[[0], ['시작일자']] # 확인용 코드

In [ ]:
df_distance.loc[[0], ['시작일자']].squeeze() # 확인용 코드

In [ ]:
df_distance.loc[[0], ['차수']] # 확인용 코드

In [ ]:
df_distance.loc[[0], ['차수']].squeeze() # 확인용 코드

In [ ]:
# date_range 생성
def date_range(start, end):
    start = datetime.datetime.strptime(start, "%Y-%m-%d")
    end = datetime.datetime.strptime(end, "%Y-%m-%d")
    dates = [date.strftime("%Y-%m-%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates

In [ ]:
# date_range범위를 고려한 새로운 데이터 프레임 생성
dates = date_range((df_distance.loc[[0], ['시작일자']]).squeeze(), (df_distance.loc[[0], ['종료일자']]).squeeze())
df_distance_date = pd.DataFrame({'날짜' : dates})
df_distance_degree = df_distance.loc[[0], ['차수']]
df_distance_piece = pd.concat([df_distance_date, df_distance_degree], axis = 1)
df_distance_new = df_distance_piece.fillna(method='ffill') # 바로 앞 데이터로 결측치 채움

In [ ]:
# date_range범위를 고려한 새로운 데이터 프레임 생성
for i in range(1, len(df_distance)):
    dates = date_range((df_distance.loc[[i], ['시작일자']]).squeeze(), (df_distance.loc[[i], ['종료일자']]).squeeze())
    df_distance_date = pd.DataFrame({'날짜' : dates})
    degree = df_distance.loc[[i], ['차수']].squeeze()
    df_distance_degree = pd.DataFrame({'차수' : [degree]})
    df_distance_piece = pd.concat([df_distance_date,df_distance_degree], axis = 1)
    df_distance_piece = df_distance_piece.fillna(method='ffill') # 바로 앞 데이터로 결측치 채움
    df_distance_new = pd.concat([df_distance_new, df_distance_piece], axis = 0)

In [ ]:
df_distance_new # 확인용 코드

In [ ]:
df_distance_new = df_distance_new.drop_duplicates(['날짜'])

In [ ]:
df_distance_new # 확인용 코드

In [ ]:
df_distance_new = df_distance_new.reset_index()
df_distance_new = df_distance_new.drop(['index'], axis = 1)

In [ ]:
df_distance_new # 확인용 코드

In [ ]:
df_distance_new.info() # 확인용 코드

In [ ]:
# 데이터 전처리 - 날짜 형식 변환
date_set=[]
remove_set = ['2020-04-15', '2020-04-30', '2020-05-01', '2020-05-05', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-09', '2020-12-25', '2020-12-31', '2021-01-01', '2021-02-11', '2021-02-12', '2021-03-01', '2021-05-05', '2021-05-19', '2021-08-16', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-04', '2021-10-11', '2021-12-31', '2022-01-31', '2022-02-01', '2022-02-02']
for item in df_distance_new['날짜']:
    break_flag = 1
    # print(item)
    item=item.split('-')
    year=int(item[0])
    month=int(item[1])
    day=int(item[2])
    for remove_data in remove_set:
        if(remove_data == str(date(year, month, day)) or date(year, month, day).weekday() > 4):
            date_set+=[np.nan]
            break_flag = 0
            break
    if(break_flag == 1):
        date_set+=[str(date(year,month,day))]
        
df_distance_new['날짜'] = date_set
df_distance_new.dropna(axis=0, inplace = True)
df_distance_new = df_distance_new.drop_duplicates(['날짜'])

df_distance_new["차수"]=df_distance_new["차수"].apply(float)

# print(type(remove_set[0]))
# print(type(str(date(year, month, day))))
# print(str(date(year, month, day)))

# dropna는 행(열) 중 에서 결측치가 존재하는 행(열)을 제거한다.
# axis = 0이면 행을 제거하고 axis = 1이면 열을 제거한다.
# thresh = N 옵션을 주면 해당 행(열)에 N개 이상의 값이 존재하면 결측치가 존재해도 행(열)을 제거하지 않는다.
# inplace = True 옵션을 주면 항구적 적용이 가능하다.

In [ ]:
df_distance_new # 확인용 코드

In [ ]:
df_distance_new.info() # 확인용 코드

In [ ]:
# 오미크론 우세종 된 이후 데이터 제거
df_distance_new = df_distance_new[df_distance_new['날짜'] < '2022-01-09']

In [ ]:
df_distance_new

In [ ]:
df_distance_new.info()

# Covid

In [ ]:
df_covid_num = df_covid.drop(['자치구 기준일'], axis = 1)

In [ ]:
df_covid_num.head()

In [ ]:
# 서울 전체구 데이터 병합
df_covid_num = df_covid_num.sum(axis = 1)

In [ ]:
df_covid_num.head()

In [ ]:
df_covid_num = df_covid_num.to_frame()
df_covid_num.columns = ['서울 전체']

In [ ]:
df_covid_num.head()

In [ ]:
df_covid = pd.concat([df_covid['자치구 기준일'], df_covid_num], axis = 1)

In [ ]:
df_covid.head()

In [ ]:
# 데이터 전처리 - 날짜 형식 변환
date_set=[]
remove_set = ['2020-04-15', '2020-04-30', '2020-05-01', '2020-05-05', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-09', '2020-12-25', '2020-12-31', '2021-01-01', '2021-02-11', '2021-02-12', '2021-03-01', '2021-05-05', '2021-05-19', '2021-08-16', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-04', '2021-10-11', '2021-12-31', '2022-01-31', '2022-02-01', '2022-02-02']
for item in df_covid['자치구 기준일']:
    break_flag = 1
    # print(item)
    item=item.split('.')
    year=int(item[0])
    month=int(item[1])
    day=int(item[2])
    for remove_data in remove_set:
        if(remove_data == str(date(year, month, day)) or date(year, month, day).weekday() > 4):
            date_set+=[np.nan]
            break_flag = 0
            break
    if(break_flag == 1):
        date_set+=[str(date(year,month,day))]
        # print(datetime.date(year,month,day))
df_covid['자치구 기준일'] = date_set
# df_covid['자치구 기준일'] = pd.to_datetime(df_covid['자치구 기준일'])

df_covid.dropna(axis=0, inplace = True)
df_covid = df_covid.drop_duplicates(['자치구 기준일'])
df_covid["서울 전체"]=df_covid["서울 전체"].apply(float)
df_covid = df_covid.sort_values('자치구 기준일', ascending = True)

# print(type(remove_set[0]))
# print(type(str(date(year, month, day))))
# print(str(date(year, month, day)))

# dropna는 행(열) 중 에서 결측치가 존재하는 행(열)을 제거한다.
# axis = 0이면 행을 제거하고 axis = 1이면 열을 제거한다.
# thresh = N 옵션을 주면 해당 행(열)에 N개 이상의 값이 존재하면 결측치가 존재해도 행(열)을 제거하지 않는다.
# inplace = True 옵션을 주면 항구적 적용이 가능하다.

In [ ]:
# 확인용 코드
print(len(remove_set))

In [ ]:
# 확인용 코드
df_covid['자치구 기준일']

In [ ]:
# 확인용 코드
df_covid.head(3)

In [ ]:
# 확인용 코드
df_covid.tail(3)

In [ ]:
# 확인용 코드
df_covid.info()

In [ ]:
# 오미크론 우세종 된 이후 데이터 제거
df_covid = df_covid[df_covid['자치구 기준일'] < '2022-01-09']

In [ ]:
df_covid

In [ ]:
df_covid.info()

# Stock

In [ ]:
# 확인용 코드
df_stock.head()

In [ ]:
# 데이터 전처리 - 전치 + 날짜 형식 변환
df_stock_day = df_stock.columns.values.tolist()
df_stock = df_stock.T
# print(df_stock.T.head())
df_stock['날짜'] = df_stock_day

In [ ]:
date_set=[]
for item in df_stock['날짜']:
    # print(item)
    if(item == '계정별'):
        date_set+=[np.nan]
    else:
        item=item.split('.')
        year=int(item[0])
        month=int(item[1])
        day=int(item[2])
        date_set+=[str(date(year,month,day))]
    # print(datetime.date(year,month,day))
df_stock['날짜'] = date_set
# df_stock['날짜'] = pd.to_datetime(df_stock['날짜'])

df_stock.dropna(axis=0, inplace = True)
df_stock.rename(columns={0:'KOSPI'}, inplace = True)
df_stock = df_stock[['KOSPI', '날짜']]
df_stock["KOSPI"]=df_stock["KOSPI"].apply(float)
df_stock = df_stock.sort_values('날짜', ascending = True)

In [ ]:
# 확인용 코드
df_stock.head(3)

In [ ]:
# 확인용 코드
df_stock.tail(3)

In [ ]:
df_stock.info()

In [ ]:
df_stock = df_stock[df_stock['날짜'] < '2022-01-09']

In [ ]:
df_stock.info()

# Vaccine

In [ ]:
# 확인용 코드
df_vaccine.head()

In [ ]:
# 데이터 전처리 - 필요한 열 추출
df_vaccine = df_vaccine[['접종일', '당일 1차접종자 수', '1차접종 누계', '당일 2차접종자 수', '2차접종 누계', '당일 추가접종자 수', '추가접종 누계']]
df_vaccine = df_vaccine.fillna(0)

In [ ]:
# 데이터 전처리 - 날짜 형식 변환
date_set=[]
for item in df_vaccine['접종일']:
    # print(item)
    item=item.split('.')
    year=int(item[0])
    month=int(item[1])
    day=int(item[2])

    date_set+=[str(date(year,month,day))]
    # print(datetime.date(year,month,day))
df_vaccine['접종일'] = date_set
# df_covid['자치구 기준일'] = pd.to_datetime(df_covid['자치구 기준일'])

df_vaccine.dropna(axis=0, inplace = True)

In [ ]:
# 확인용 코드
df_vaccine.head(3)

In [ ]:
# 확인용 코드
df_vaccine.tail(3)

In [ ]:
# 다른 데이터와 날짜 맞추기 위해 평일 날짜 생성 + 자료형 변환
df_hole_day = pd.DataFrame(pd.date_range('20200205', datetime.date(2021,4,20), freq='B'))
df_hole_day.columns=['접종일']
df_hole_day = df_hole_day.astype({'접종일':'str'})
df_hole_day["당일 1차접종자 수"] = 0
df_hole_day["1차접종 누계"] = 0
df_hole_day["당일 2차접종자 수"] = 0
df_hole_day["2차접종 누계"] = 0
df_hole_day["당일 추가접종자 수"] = 0
df_hole_day["추가접종 누계"] = 0

In [ ]:
# 확인용 코드
df_hole_day

In [ ]:
# 데이터 합병
df_hole_day
df_vaccine = pd.concat([df_vaccine, df_hole_day])

In [ ]:
# 확인용 코드
df_vaccine.head(3)

In [ ]:
# 확인용 코드
df_vaccine.tail(3)

In [ ]:
# 확인용 코드
df_vaccine.info()

In [ ]:
# 데이터 전처리 - 날짜 형식 변환
remove_set = ['2020-04-15', '2020-04-30', '2020-05-01', '2020-05-05', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-09', '2020-12-25', '2020-12-31', '2021-01-01', '2021-02-11', '2021-02-12', '2021-03-01', '2021-05-05', '2021-05-19', '2021-08-16', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-04', '2021-10-11', '2021-12-31', '2022-01-31', '2022-02-01', '2022-02-02']
date_set=[]
for item in df_vaccine['접종일']:
    # print(item)
    break_flag = 1
    item=item.split('-')
    year=int(item[0])
    month=int(item[1])
    day=int(item[2])

    for remove_data in remove_set:
        if(remove_data == str(date(year, month, day)) or date(year, month, day).weekday() > 4 or str(date(year,month,day)) > '2022-02-17'):
            date_set+=[np.nan]
            break_flag = 0
            break
    if(break_flag == 1):
        date_set+=[str(date(year,month,day))]
        # print(datetime.date(year,month,day))
df_vaccine['접종일'] = date_set
# df_covid['자치구 기준일'] = pd.to_datetime(df_covid['자치구 기준일'])

In [ ]:
# 데이터 전처리 - 결측치, 중복값 제거 + 정렬
df_vaccine.dropna(axis=0, inplace = True)
df_vaccine = df_vaccine.drop_duplicates(['접종일'])
df_vaccine = df_vaccine.sort_values('접종일', ascending = True)

In [ ]:
# 확인용 코드
df_vaccine.head(3)

In [ ]:
# 확인용 코드
df_vaccine.tail(3)

In [ ]:
# 확인용 코드
df_vaccine.info()

In [ ]:
df_vaccine = df_vaccine[df_vaccine['접종일'] < '2022-01-09']

In [ ]:
df_vaccine.info()

In [ ]:
# 데이터 전처리 - 인덱스 재할당
df_vaccine = df_vaccine.reset_index()

In [ ]:
# 확인용 코드
df_vaccine

# 차집합 확인

In [ ]:
# 확인용 코드
list1 = list(np.array(df_covid['자치구 기준일'].tolist()))
print(len(set(list1)))

In [ ]:
# 확인용 코드
list2 = list(np.array(df_stock['날짜'].tolist()))
print(len(set(list2)))

In [ ]:
# 확인용 코드
list3 = list(np.array(df_vaccine['접종일'].tolist()))
list4 = list(np.array(df_vaccine['당일 1차접종자 수'].tolist()))
list5 = list(np.array(df_vaccine['1차접종 누계'].tolist()))
list6 = list(np.array(df_vaccine['당일 2차접종자 수'].tolist()))
list7 = list(np.array(df_vaccine['2차접종 누계'].tolist()))
list8 = list(np.array(df_vaccine['당일 추가접종자 수'].tolist()))
list9 = list(np.array(df_vaccine['추가접종 누계'].tolist()))
print(len(set(list3)))
print(len(set(list4)))
print(len(set(list5)))
print(len(set(list6)))
print(len(set(list7)))
print(len(set(list8)))
print(len(set(list9)))

In [ ]:
# 확인용 코드
A = list(set(list3) - set(list2))
B = list(set(list3) - set(list1))
print(A)
print(B)

In [ ]:
# 확인용 코드
df_vaccine

In [ ]:
# 확인용 코드
df_stock.info()

In [ ]:
# 확인용 코드
df_covid.info()

In [ ]:
# 확인용 코드
df_covid.head(5)

In [ ]:
# 확인용 코드
df_covid.tail(5)

In [ ]:
# 확인용 코드
df_stock.head(5)

In [ ]:
# 확인용 코드
df_stock.tail(5)

In [ ]:
# 확인용 코드
df_vaccine.head(5)

In [ ]:
# 확인용 코드
df_vaccine.tail(5)

In [ ]:
# 확인용 코드
df_distance_new.head(5)

In [ ]:
# 확인용 코드
df_distance_new.tail(5)

# 데이터 병합

In [ ]:
# 데이터 전처리 - 인덱스 재할당
df_vaccine = df_vaccine.reset_index()
df_covid = df_covid.reset_index()
df_distance_new = df_distance_new.reset_index()

In [ ]:
# 데이터 병합
df_input = pd.concat([df_vaccine, df_covid['서울 전체']], axis=1)
df_input = pd.concat([df_input, df_distance_new['차수']], axis=1)

In [ ]:
df_input # 확인용 코드

In [ ]:
df_input.info() # 확인용 코드

In [ ]:
# 필요없는 열 삭제
df_input = df_input.drop(['level_0', 'index', '접종일'], axis=1)

In [ ]:
df_input.info() # 확인용 코드

In [ ]:
df_input # 확인용 코드

In [ ]:
df_input.head() # 확인용 코드

In [ ]:
df_input.tail() # 확인용 코드

# 시각화

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# 시각화
df_covid_count = df_covid['서울 전체']
df_kospi = df_stock['KOSPI']
plt.scatter(df_covid_count, df_kospi, label='Original Data')
plt.xlabel('covid count')
plt.ylabel('kospi')
plt.legend(loc='lower right')
plt.show()

# 데이터셋 분류

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터셋 분류
train_input, test_input, train_target, test_target = train_test_split(
    df_input, df_kospi,  random_state=42)

In [ ]:
# 확인용 코드
print(train_input.shape, test_input.shape)
print(train_target.shape, test_target.shape)

## 사이킷런의 변환기

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# 변환기
poly = PolynomialFeatures(include_bias=False)

# 특성 조합 찾기
poly.fit(train_input)

# 실제 특성 조합 데이터 반환
train_poly = poly.transform(train_input)

In [ ]:
# 확인용 코드
print(train_poly.shape)

In [ ]:
# 특성 조합 확인
poly.get_feature_names()

In [ ]:
test_poly = poly.transform(test_input)

In [ ]:
# 확인용 코드
print(train_poly.shape)
print(train_target.shape)

# SVM

In [ ]:
train_input, test_input, train_target, test_target = train_test_split(
    df_input, df_kospi,  random_state=42)

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

sc = RobustScaler()

train_input_std = sc.fit(train_input)

train_input_std = sc.transform(train_input)
test_input_std = sc.transform(test_input)

train_input_scaled = pd.DataFrame(train_input_std, columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'])
test_input_scaled = pd.DataFrame(test_input_std, columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'])

svm_confirmed = SVR(shrinking=True, kernel='rbf',gamma=0.00001, epsilon=1,degree=5, C=100)
svm_confirmed.fit(train_input_scaled, train_target)

x = np.arange(0, 250000).reshape(-1, 1)

   

svm_test_pred = svm_confirmed.predict(test_input_scaled)
print('MAE:', mean_absolute_error(svm_test_pred, test_target))
print('MSE:',mean_squared_error(svm_test_pred, test_target))

print(svm_confirmed.score(train_input_scaled, train_target))
print(svm_confirmed.score(test_input_scaled, test_target))

In [ ]:
train_input

In [ ]:
list(np.logspace(-1, 1,num = 10, base = 10))

In [ ]:
svm = SVR(shrinking=True, kernel='rbf')
param_grid = {"gamma":[1000, 100, 10, 1, 0.1, 0.01, 0.001],
             "C":[100, 300, 1000, 5000, 10000, 30000, 50000],
             "epsilon": [0.000001, 0.00001, 0.0001, 0.001, 0.01],
             "degree":[1, 2, 3, 4, 5]}
cv=KFold(n_splits=10)
gs_svm = GridSearchCV(svm, param_grid=param_grid, cv=cv)

gs_svm.fit(train_input_scaled, train_target)
# y_predict = best_model.predict(x_test)

In [ ]:
svm_df = pd.DataFrame(gs_svm.cv_results_)
svm_df

In [ ]:
print('svm best params', gs_svm.best_params_)   # 최적의 파라미터 값 출력
print('svm best score', gs_svm.best_score_)      # 최고의 점수

best_model_svm = gs_svm.best_estimator_

# RF

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

In [ ]:
print(df_input.columns)
dataNames = df_input.columns

In [ ]:
train_input, test_input, train_target, test_target = train_test_split(df_input, df_kospi, test_size = 0.25, random_state = 531)


from sklearn.preprocessing import MinMaxScaler

s = MinMaxScaler()

train_input_std = s.fit(train_input)

train_input_std = s.transform(train_input)
test_input_std = s.transform(test_input)

train_input_scaled = pd.DataFrame(train_input_std, columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'])
test_input_scaled = pd.DataFrame(test_input_std, columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'])


print(len(train_input_scaled), len(test_input_scaled))
mseOos = []
nTreeList = range(50, 500, 10)

for iTrees in nTreeList:
    depth = None
    maxFeat = 4 #조정해볼 것
    RFModel = ensemble.RandomForestRegressor(n_estimators=iTrees,
                    max_depth=depth, max_features=maxFeat,
                    oob_score=False, random_state=531)
    RFModel.fit(train_input_scaled, train_target)
    #데이터 세트에 대한 MSE 누적
    prediction = RFModel.predict(test_input_scaled)
    mseOos.append(mean_squared_error(test_target, prediction))
print("MSE")
print(mseOos)

In [ ]:
print(RFModel.score(train_input_scaled, train_target))
print(RFModel.score(test_input_scaled, test_target))

In [ ]:
plt.plot(nTreeList, mseOos)
plt.xlabel('Number of Trees in Ensemble')
plt.ylabel('Mean Squared Error')
#plot.ylim([0.0, 1.1*max(mseOob)])
plt.show()

#피처 중요도 도표 그리기
featureImportance = RFModel.feature_importances_

#가장 높은 중요도 기준으로 스케일링
featureImportance = featureImportance/featureImportance.max()
sorted_idx = np.argsort(featureImportance)
barPos = np.arange(sorted_idx.shape[0])+.5
plt.barh(barPos, featureImportance[sorted_idx], align='center')
plt.yticks(barPos, dataNames[sorted_idx])
plt.xlabel('Variable Importance')
plt.show()

In [ ]:
rf = ensemble.RandomForestRegressor(oob_score=False, random_state=531)
param_grid = {"n_estimators":[5, 10, 30, 50, 100],
             "max_depth":[2, 4, 6, 8, 10, 12],
             "max_features": [0, 1, 2, 3, 4, 5, 6, 7]}
cv=KFold(n_splits=10)
gs_rf = GridSearchCV(rf, param_grid=param_grid, cv=cv)

gs_rf.fit(train_input_scaled, train_target)
# y_predict = best_model.predict(x_test)

In [ ]:
rf_df = pd.DataFrame(gs_rf.cv_results_)
rf_df

In [ ]:
print('rf best params', gs_rf.best_params_)   # 최적의 파라미터 값 출력
print('rf best score', gs_rf.best_score_)      # 최고의 점수

best_model_rf = gs_rf.best_estimator_

## 임의 값 대입

In [ ]:
import pickle
from sklearn.externals import joblib

In [ ]:
ko_max = 0.0
x2 = 0
x4 = 0
x6 = 0

for x1 in range(0, 80000, 500):
    x4 = 0
    x2 += x1
    for x3 in range(0, 90000, 500):
        x6 = 0
        x4 += x3 
        if x2 < x4:
            break
        x5 = 0
        x6 = 0
        for x7 in range(0, 60000, 500):
            for x8 in range(0, 5):
                ko = RFModel.predict([[x1, x2, x3, x4, x5, x6, x7, x8]])
                if ko_max < ko:
                    ko_max = ko
                    x_list = [x1, x2, x3, x4, x5, x6, x7, x8]

print(ko)
print(x_list)